In [ ]:
import os
import re
import glob
import pandas as pd
import csv
from sklearn.cross_decomposition import CCA
from sklearn.utils import resample
from sklearn.utils import shuffle
import load_approved_data
import matplotlib.pyplot as plt
import numpy as np
import time

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
bootstrap_iters = 500
n_cca_components = 50

In [ ]:
#read in standard demographic data to pandas dataframe
standard_data_filename = r'camcan_civet_ratings_combined.csv'
standard_col_list = ['CCID','Age','Sex','Rating']
standard_data = pd.read_csv(standard_data_filename, usecols=standard_col_list, index_col='CCID')

#Change variable type - likely should move this later
standard_data['Sex'] = standard_data['Sex'].astype('category')
standard_data['Sex'] = standard_data['Sex'].cat.codes

#filter out any columns which didn't meet approved rating
#and then remove the rating column
standard_data_filtered = standard_data[standard_data.Rating >= 0.5]
standard_data_filtered = standard_data_filtered.drop(['Rating'], axis=1)
#print (standard_data_filtered.head(5))

#filter out any subjects under the age of 40
standard_data_filtered = standard_data_filtered[standard_data_filtered.Age >= 40]

In [ ]:
#read in other demographic data to pandas dataframe
approved_data_filename = r'Cam-CAN\approved_data.tsv'
#approved_col_list = ['CCID', 'homeint_handedness', 'additional_qual','homeint_v349']

#approved_data = pd.read_csv(approved_data_filename, usecols=approved_col_list,  index_col='CCID', sep='\t')
approved_data = load_approved_data.load_approved_data(approved_data_filename)

#print (approved_data.head(5))

#merge data into one dataframe
demographics = pd.merge(standard_data_filtered, approved_data, how='left', left_on='CCID', right_on='CCID')

In [ ]:
all_ccids = demographics.index.tolist()
print(len(all_ccids))
sample_ccids = resample(all_ccids, replace=True, random_state=0)
print(len(sample_ccids))

In [ ]:
### TEMP ####
# Remove these variables until figure out how to use them
demographics.drop(['homeint_v96'], inplace=True, axis=1)
demographics.drop(['homeint_v363'], inplace=True, axis=1)
n_comp = demographics.shape[0]

In [ ]:
print(demographics.columns)

In [ ]:
print(demographics['Sex'].value_counts())

In [ ]:
print(len(approved_data.index))

In [ ]:
#read in pca of vertices
#vertexes_data_filename = r'Cam-CAN\civet\output\output\T1w_surface_rsl_left_native_volume_40mm_dataframe_comp.csv'
component_file_location = r'C:\Users\grace\Documents\School\Grad\CoBrA\Cam-CAN\civet21\native_rms_rsl_tlaplace_30mm_AAL.csv'
#component_file_location = r'C:\Users\grace\Documents\School\Grad\CoBrA\Cam-CAN\civet21\native_rms_rsl_tlaplace_30mm_left_ica_100.csv'

vertexes = pd.read_csv(component_file_location, header=None)#.transpose()

In [ ]:
print(vertexes.shape)
print(demographics.shape)
num_participants = demographics.shape[0]

In [ ]:
cca_model = CCA(n_components=50)
cca_model.fit(demographics,vertexes)
demographics_T, vertexes_T = cca_model.transform(demographics,vertexes)
result = np.corrcoef(demographics_T.T, vertexes_T.T).diagonal(offset=cca_model.n_components)
cc_corr = np.corrcoef(cca_model.x_scores_, rowvar=False).diagonal(offset=cca_model.n_components)

In [ ]:
print(result)

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.plot(np.arange(cca_model.n_components)+1, result, 'ko')
plt.xlim(.5, .5+cca_model.n_components)
plt.xticks(np.arange(cca_model.n_components)+1)
plt.xlabel('Canonical component')
plt.ylabel('Canonical correlation')
plt.title('Canonical correlations')

In [ ]:
print(cc_corr)

In [ ]:

#X = cca_model.predict(demographics)
X_weights = cca_model.x_weights_
X_loadings = cca_model.x_loadings_
Y_weights = cca_model.y_weights_
Y_loadings = cca_model.y_loadings_
coefficients = cca_model.coef_

my_coef_counts = dict()
my_x_loadings = dict()
for x_load in X_loadings.transpose():
    print(x_load)
    my_x_loadings[repr(x_load)] = 1
for coef in coefficients:
    #print(coef.shape)
    #Note: use repr to convert coef to string since can't hash numpy array
    my_coef_counts[repr(coef)] = 1 

In [ ]:
print(coefficients.shape)
print(X_loadings.shape)
print(len(my_x_loadings))

In [ ]:
vertexes_shuffled = shuffle(vertexes, random_state=1)
#vertexes_shuffled = shuffle(vertexes)
cca_model_shuffled = CCA(n_components=50)
cca_model_shuffled.fit(demographics,vertexes_shuffled)
coefficients_temp = cca_model_shuffled.coef_
X_loadings_temp = cca_model_shuffled.x_loadings_
Y_loadings_temp = cca_model_shuffled.y_loadings_
demographics_s_T, vertexes_s_T = cca_model_shuffled.transform(demographics,vertexes_shuffled)
result_shuffled = np.corrcoef(demographics_s_T.T, vertexes_s_T.T).diagonal(offset=cca_model_shuffled.n_components)


In [ ]:
print(result_shuffled)

In [ ]:
print(testcorrs)

In [ ]:
corrcoefs_bstrap = np.zeros((n_cca_components, bootstrap_iters))
print(corrcoefs_bstrap.shape)
fwe_vals = np.zeros((n_cca_components,1))

In [ ]:
demographics_shuffled, vertexes_shuffled = shuffle(demographics, vertexes, random_state=2)
demographics_shuffled_split1 = demographics_shuffled[:num_participants//2]
vertexes_shuffled_split1 = vertexes_shuffled[:num_participants//2]
demographics_shuffled_split2 = demographics_shuffled[num_participants//2:]
vertexes_shuffled_split2 = vertexes_shuffled[num_participants//2:]

In [ ]:
demographics_shuffled_split1

In [ ]:
demographics_shuffled_split2

In [ ]:
cca_model_shuffled_split2 = CCA(n_components=50)
cca_model_shuffled_split2.fit(demographics_shuffled_split2,vertexes_shuffled_split2)
coefficients_temp = cca_model_shuffled_split2.coef_
X_loadings_temp_split2 = cca_model_shuffled_split2.x_loadings_
Y_loadings_temp_split2 = cca_model_shuffled_split2.y_loadings_
X_weights_temp_split2 = cca_model_shuffled_split2.x_weights_
Y_weights_temp_split2 = cca_model_shuffled_split2.y_weights_
test1_c, test2_c = cca_model_shuffled_split2.transform(demographics_shuffled_split2, vertexes_shuffled_split2)
testcorrs_split2 = np.corrcoef(test1_c.T, test2_c.T).diagonal(offset=cca_model_shuffled_split2.n_components)

In [ ]:
cca_model_shuffled_split1 = CCA(n_components=50)
cca_model_shuffled_split1.fit(demographics_shuffled_split1,vertexes_shuffled_split1)
coefficients_temp = cca_model_shuffled_split1.coef_
X_loadings_temp_split1 = cca_model_shuffled_split1.x_loadings_
Y_loadings_temp_split1 = cca_model_shuffled_split1.y_loadings_
X_weights_temp_split1 = cca_model_shuffled_split1.x_weights_
Y_weights_temp_split1 = cca_model_shuffled_split1.y_weights_
test1_c, test2_c = cca_model_shuffled_split1.transform(demographics_shuffled_split1, vertexes_shuffled_split1)
testcorrs_split1 = np.corrcoef(test1_c.T, test2_c.T).diagonal(offset=cca_model_shuffled_split1.n_components)

In [ ]:
print(np.linalg.norm(X_loadings_temp_split2-X_loadings_temp_split1))
print(np.linalg.norm(Y_loadings_temp_split2-Y_loadings_temp_split1))

In [ ]:
print(np.linalg.norm(X_weights_temp_split2-X_weights_temp_split1))
print(np.linalg.norm(Y_weights_temp_split2-Y_weights_temp_split1))

In [ ]:
def levenshtein(a,b):
    "Calculates the Levenshtein distance between a and b."
    n, m = len(a), len(b)
    if n > m:
        # Make sure n <= m, to use O(min(n,m)) space
        a,b = b,a
        n,m = m,n
        
    current = range(n+1)
    for i in range(1,m+1):
        previous, current = current, [i]+[0]*n
        for j in range(1,n+1):
            add, delete = previous[j]+1, current[j-1]+1
            change = previous[j-1]
            if a[j-1] != b[i-1]:
                change = change + 1
            current[j] = min(add, delete, change)
            
    return current[n]

In [ ]:
print(testcorrs_split1)
print(testcorrs_split2)

In [ ]:

#vertexes_shuffled = shuffle(vertexes)
cca_model_shuffled = CCA(n_components=50)
cca_model_shuffled.fit(demographics,vertexes_shuffled)
coefficients_temp = cca_model_shuffled.coef_
X_loadings_temp = cca_model_shuffled.x_loadings_
Y_loadings_temp = cca_model_shuffled.y_loadings_
test1_c, test2_c = cca_model_shuffled.transform(demographics, vertexes_shuffled)
testcorrs = np.corrcoef(test1_c.T, test2_c.T).diagonal(offset=cca_model_shuffled.n_components)
corrcoefs_bstrap[:,b_iter] = testcorrs
"""
for coef in coefficients_temp:
    coef_hashable = repr(coef)
    if coef_hashable in my_coef_counts:
        my_coef_counts[coef_hashable] += 1
    else:
        my_coef_counts[coef_hashable] = 1
for x_load in X_loadings_temp.transpose():
    x_load_hashable = repr(x_load)
    if x_load_hashable in my_x_loadings:
        my_x_loadings[x_load_hashable] += 1
    else:
        my_x_loadings[x_load_hashable] = 1
"""
if b_iter % 100 == 0:
    print(str(b_iter) + " at " + time.ctime(time.time()))

In [ ]:
print(corrcoefs_bstrap.shape)

In [ ]:
for comp_x in range(0,n_cca_components):
    corr_x = result[comp_x]
    temp_compare = np.zeros(bootstrap_iters)
    for b_iter in range(0,bootstrap_iters):
        if corr_x < corrcoefs_bstrap[0,b_iter]:
            temp_compare[b_iter] = 1
        else:
            temp_compare[b_iter] = 0
    fwe_vals[comp_x] = (1+sum(temp_compare))/bootstrap_iters
    #print(temp_compare)
    #print(corr_x, fwe_vals[comp_x])
print(fwe_vals)

In [ ]:
print(corrcoefs_bstrap)
print(corrcoefs_bstrap.shape)
print(corrcoefs_bstrap[0])

In [ ]:
def get_key(my_dict, val):
    for key, value in my_dict.items():
         if val == value:
             return key
 
    return "key doesn't exist"

In [ ]:
print(max(my_coef_counts.values()))
#print(get_key(my_coef_counts,2))

In [ ]:
print(max(my_x_loadings.values()))
#print(get_key(my_coef_counts,2))

In [ ]:
print(demographics_shuffled)

In [ ]:
print(coefficients.shape)
print()

In [ ]:
total_times_pc_selected = np.zeros(vertexes.shape[1])
abs_val_pc_selected = np.zeros(vertexes.shape[1])
total_times_dem_selected = np.zeros(demographics.shape[1])
abs_val_dem_selected = np.zeros(demographics.shape[1])

In [ ]:
abs_val_dem_selected = np.sum(np.absolute(X_loadings),axis=1)
abs_val_pc_selected = np.sum(np.absolute(Y_loadings),axis=1)

In [ ]:
plt.bar(list(range(0,63)), abs_val_dem_selected)
plt.show()
plt.bar(list(range(0,100)), abs_val_pc_selected)
plt.show()

In [ ]:
#print(max(X_weights[:,0]))
#print(X_weights[:,0])
#print(X_loadings[:,0])
#print(max(X_loadings[:,0]))
#print(X_weights[0,0])

In [ ]:
print(max(X_loadings[:,0]))
print(max(X_loadings[:,0]))

for ix in range(0,len(X_weights[:,0])):
    val = X_weights[ix,0]
    if val > 0.1 or val <-0.1:
        print(ix)
        print(demographics.columns[ix] + " : " + str(val))

In [ ]:
print(np.argmax(X_loadings[:,0]))

In [ ]:
n_indices = 1
cc_cal = 0

In [ ]:
print("Top " + str(n_indices) + " pos weighted")
for current_cc_cal in range(0,cc_cal+1):
    X_max_idx = np.argpartition(X_loadings[:,current_cc_cal], -n_indices)[-n_indices:]
    X_max_indices = X_max_idx[np.argsort((-X_loadings[:,current_cc_cal])[X_max_idx])]
    #print(X_max_indices)
    dc_mi1 = demographics.columns[X_max_indices[0]]
    dc_mi2 = demographics.columns[X_max_indices[1]]
    dc_mi3 = demographics.columns[X_max_indices[2]]
    dc_mi4 = demographics.columns[X_max_indices[3]]

    dict_col_names = pd.read_csv(r'Cam-CAN\approved_col_headers.csv', index_col=0).T.to_dict('list')
    for key in dict_col_names:
        val_tostring_temp = dict_col_names[key][0]
        dict_col_names[key] = val_tostring_temp
    dict_col_names['Age'] = ''
    dict_col_names['Sex'] = ''

    print("CC " + str(current_cc_cal+1) + ": " + dc_mi1 + " (" + dict_col_names[dc_mi1] + "), " + dc_mi2 + " (" + dict_col_names[dc_mi2] + "), " + dc_mi3 + " (" + dict_col_names[dc_mi3] + "), " + dc_mi4 + " (" + dict_col_names[dc_mi4] + ")")

In [ ]:
print("Top " + str(n_indices) + " neg weighted")
for current_cc_cal in range(0,cc_cal+1):
    X_min_idx = np.argpartition(X_loadings[:,current_cc_cal], n_indices)[:n_indices]
    X_min_indices = X_min_idx[np.argsort((X_loadings[:,current_cc_cal])[X_min_idx])]
    #print(X_min_indices)
    
    dc_mi1 = demographics.columns[X_min_indices[0]]
    dc_mi2 = demographics.columns[X_min_indices[1]]
    dc_mi3 = demographics.columns[X_min_indices[2]]
    dc_mi4 = demographics.columns[X_min_indices[3]]

    dict_col_names = pd.read_csv(r'Cam-CAN\approved_col_headers.csv', index_col=0).T.to_dict('list')
    for key in dict_col_names:
        val_tostring_temp = dict_col_names[key][0]
        dict_col_names[key] = val_tostring_temp
    dict_col_names['Age'] = ''
    dict_col_names['Sex'] = ''

    print("CC " + str(current_cc_cal+1) + ": " + dc_mi1 + " (" + dict_col_names[dc_mi1] + "), " + dc_mi2 + " (" + dict_col_names[dc_mi2] + "), " + dc_mi3 + " (" + dict_col_names[dc_mi3] + "), " + dc_mi4 + " (" + dict_col_names[dc_mi4] + ")")

In [ ]:
print("Top " + str(n_indices) + " pos weighted")
for current_cc_cal in range(0,cc_cal+1):
    Y_max_idx = np.argpartition(Y_loadings[:,current_cc_cal], -n_indices)[-n_indices:]
    Y_max_indices = Y_max_idx[np.argsort((-Y_loadings[:,current_cc_cal])[Y_max_idx])]
    print("CC " + str(current_cc_cal+1) + ": " + str(Y_max_indices))

In [ ]:
print("Top " + str(n_indices) + " neg weighted")
for current_cc_cal in range(0,cc_cal+1):
    Y_min_idx = np.argpartition(Y_loadings[:,current_cc_cal], n_indices)[:n_indices]
    Y_min_indices = Y_min_idx[np.argsort((Y_loadings[:,current_cc_cal])[Y_min_idx])]
    print("CC " + str(current_cc_cal+1) + ": " + str(Y_min_indices))

In [ ]:
Y_max_idx = np.argpartition(Y_loadings[:,cc_cal], -n_indices)[-n_indices:]
Y_max_indices = Y_max_idx[np.argsort((-Y_loadings[:,cc_cal])[Y_max_idx])]
print(Y_max_indices)

In [ ]:
Y_min_idx = np.argpartition(Y_loadings[:,cc_cal], n_indices)[:n_indices]
Y_min_indices = Y_min_idx[np.argsort((Y_loadings[:,cc_cal])[Y_min_idx])]
print(Y_min_indices)

In [ ]:
max1 = max(X_loadings[:,0])
max2 = max(X_loadings[:,0].delete(max1))
max3 = max(X_loadings[:,0].delete(max1))
print(X_loadings[:,0].shape)

In [ ]:
print(Y_weights.shape)
print(max(Y_loadings[:,0]))

In [ ]:
zxcv = "Age Sex Handedness HouseholdSize SocialLife1 SocialLife2 SocialLife3 SocialLife4 SocialLife5 SocialLife6 SocialLife7 SocialLife8 SocialLife9 HearingDifficulties1 HearingDifficulties2 HearingDifficulties3 BloodPressure1 BloodPressur2 BloodPressure3 BloodPressure4 BloodPressure5 BloodPressure6 HighCholesterol1 HighCholesterol2 Angina1 Angina2 HeartAttack1 HeartAttack2 CardiacIssues1 CardiacIssues2 VaricoseVeins1 VaricoseVeins2 Migraines1 Migraines2 Stroke1 Stroke2 PulmonaryEmbolism1 PulmonaryEmbolism2 DeepVeinThrombosis1 DeepVeinThrombosis2 VascularDisease1 VascularDisease2 Diabetes1 Diabetes2 Anxiety HeadInjury1 HeadInjury2 HeadInjury3 HeadInjury4 SkullFracture AdDiagnosis MMSEScore PhysicalActivityHrs PhysicalActivityEnergy PhysicalActivityTotal PhysicalActivityOther Smoking Alcohol Depression1 Depression2 Education SocialClass EconomicGroup"
listzxcv = zxcv.split(" ")
print(listzxcv)
print(len(listzxcv))

In [ ]:
print(demographics.columns)

In [ ]:
plt.bar(list(range(0,100)), Y_loadings[:,0])
counter = 0
for y_load in Y_loadings[:,0]:
    counter +=1 
    if y_load > 0.15:
        print(counter)

In [ ]:
# Initialize number of samples
nSamples = 1000

# Define two latent variables (number of samples x 1)
latvar1 = np.random.randn(nSamples,)
latvar2 = np.random.randn(nSamples,)

# Define independent components for each dataset (number of observations x dataset dimensions)
indep1 = np.random.randn(nSamples, 4)
indep2 = np.random.randn(nSamples, 5)

# Create two datasets, with each dimension composed as a sum of 75% one of the latent variables and 25% independent component
data1 = 0.25*indep1 + 0.75*np.vstack((latvar1, latvar2, latvar1, latvar2)).T
data2 = 0.25*indep2 + 0.75*np.vstack((latvar1, latvar2, latvar1, latvar2, latvar1)).T

# Split each dataset into two halves: training set and test set
train1 = data1[:nSamples//2]
train2 = data2[:nSamples//2]
test1 = data1[nSamples//2:]
test2 = data2[nSamples//2:]

# Create a cca object as an instantiation of the CCA object class. 
nComponents = 4
cca_l = CCA(n_components=nComponents)

# Use the train() method to find a CCA mapping between the two training sets.
cca_l.fit(train1, train2)

# Use the validate() method to test how well the CCA mapping generalizes to the test data.
# For each dimension in the test data, correlations between predicted and actual data are computed.
test1_c, test2_c = cca_l.transform(train1, train2)
testcorrs = np.corrcoef(test1_c.T, test2_c.T).diagonal(offset=nComponents)


In [ ]:
plt.bar(list(range(0,63)), X_loadings[:,0])
plt.show()
plt.bar(list(range(0,63)), X_loadings[:,1])
plt.show()
plt.bar(list(range(0,63)), X_loadings[:,2])
plt.show()
plt.bar(list(range(0,63)), X_loadings[:,3])
plt.show()
plt.bar(list(range(0,63)), X_loadings[:,4])
plt.show()
plt.bar(list(range(0,63)), X_loadings[:,5])
plt.show()
plt.bar(list(range(0,63)), X_loadings[:,6])
plt.show()
plt.bar(list(range(0,63)), X_loadings[:,7])
plt.show()
plt.bar(list(range(0,63)), X_loadings[:,8])
plt.show()
plt.bar(list(range(0,63)), X_loadings[:,9])
plt.show()

In [ ]:
plt.bar(list(range(0,18)), Y_loadings[:,0])
plt.show()
plt.bar(list(range(0,18)), Y_loadings[:,1])
plt.show()
plt.bar(list(range(0,18)), Y_loadings[:,2])
plt.show()
plt.bar(list(range(0,18)), Y_loadings[:,3])
plt.show()
plt.bar(list(range(0,18)), Y_loadings[:,4])
plt.show()

In [ ]:
plt.bar(list(range(0,18)), Y_loadings[:,5])
plt.show()
plt.bar(list(range(0,18)), Y_loadings[:,6])
plt.show()
plt.bar(list(range(0,18)), Y_loadings[:,7])
plt.show()
plt.bar(list(range(0,18)), Y_loadings[:,8])
plt.show()
plt.bar(list(range(0,18)), Y_loadings[:,9])
plt.show()

In [ ]:
print(Y_loadings[:,0].shape)
plt.bar(list(range(0,100)), Y_loadings[:,0])
plt.show()

In [ ]:
import random
tempyvals = []
for i in range(0,63):
    n = random.uniform(-4, 4)
    tempyvals.append(n)
tempyvals[3] = 0
tempyvals[10] = 0
tempyvals[11] = 0
tempyvals[12] = 0
tempyvals[28] = 0
tempyvals[30] = 0
tempyvals[42] = 0
tempyvals[59] = 0
plt.bar(list(range(0,63)), tempyvals)
plt.show()

In [ ]:
with open(r'Cam-CAN\approved_col_headers.csv', newline='') as f:
        reader = csv.reader(f, delimiter=',')
        names_of_demos = [row[1].replace(" ", "") for row in reader if row and row[1]!=""]
names_of_demos = ['Age', 'Sex'] + names_of_demos[1:]
print(names_of_demos)
print(demographics.columns)
print(len(names_of_demos))
print(demographics.columns.shape)

In [ ]:
sorted_headers_positive = [x for bleep,x in sorted(zip(X_loadings[:,0],listzxcv)) if bleep >= 0][::-1]
print(sorted_headers_positive)
sorted_headers_negative = [x for bleep,x in sorted(zip(X_loadings[:,0],listzxcv)) if bleep < 0]
print(sorted_headers_negative)

In [ ]:
import heapq
largest_cc1 = heapq.nlargest(10,X_loadings[:,0])
cc1_idx1 = np.where(X_loadings[:,0] == largest_cc1[0])
print(largest_cc1)
print(demographics.columns[cc1_idx1])
print('------')
smallest_cc1 = heapq.nsmallest(10,X_loadings[:,0])
cc1_idxneg1 = np.where(X_loadings[:,0] == smallest_cc1[0])
cc1_idxneg2 = np.where(X_loadings[:,0] == smallest_cc1[1])
print(smallest_cc1)
print(demographics.columns[cc1_idxneg1])
print(demographics.columns[cc1_idxneg2])